# Building Durable AI Agents with the Agentic Loop

In this notebook, we will:
- Understand the agentic loop pattern for AI agents
- Build a travel booking agent that can dynamically select and execute tools
- See how Temporal makes AI agents reliable and observable
- Explore how agents make decisions and maintain context

## What is an Agentic Loop?

The agentic loop is a pattern where an AI agent:
1. **Observes** the current state and context
2. **Decides** what action to take next using an LLM
3. **Acts** by executing the chosen tool
4. **Updates** its context with the result
5. **Repeats** until the goal is achieved

This creates a dynamic, goal-driven system that can adapt to different scenarios.

## The Challenge: AI Agents are Unreliable

**Common problems with AI agents:**
- LLM API calls timeout or get rate limited
- Tool executions fail midway through
- Agents lose context when processes crash
- No visibility into what the agent is doing
- Expensive LLM calls get repeated unnecessarily

**What we need:** A way to make AI agents as reliable as traditional software systems.

## The Solution: Durable Execution with Temporal

Temporal provides:
- **Automatic retries** for failed LLM calls and tool executions
- **State persistence** across failures
- **Full observability** of the agent's decision-making process
- **Guaranteed completion** even with crashes

Let's build a travel booking agent that demonstrates these capabilities!

## Our Travel Booking Agent

We'll build an agent that can:
- Search for flights
- Check seat availability
- Calculate costs
- Book flights
- Send confirmations

The agent will use an LLM to decide which tools to use based on the user's request.

## Step 1: Define Our Data Models

We'll use dataclasses to define our agent's structure and tool definitions.

In [ ]:
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class ToolArgument:
    """Defines an argument for a tool."""
    name: str
    type: str
    description: str

@dataclass
class ToolDefinition:
    """Defines a tool the agent can use."""
    name: str
    description: str
    arguments: List[ToolArgument]

@dataclass
class AgentGoal:
    """Defines the agent's goal and available tools."""
    agent_name: str
    tools: List[ToolDefinition]
    description: str
    starter_prompt: str
    example_conversation_history: Optional[str] = ""

@dataclass
class BookingResult:
    """Result of the agent's execution."""
    message: str
    steps_taken: List[str]

print("Data models defined!")

## Step 2: Create Our Travel Tools

These are the tools our agent can use to accomplish tasks.

In [ ]:
from typing import Dict, Union

async def search_flights(params: Dict[str, Union[str, int]]) -> str:
    """Search for available flights."""
    origin = params.get("origin", "Unknown")
    destination = params.get("destination", "Unknown")
    date = params.get("date", "Unknown")
    
    return f"""Found 3 flights from {origin} to {destination} on {date}:
- Flight AA123: Departs 8:00 AM, arrives 8:00 PM, $450
- Flight UA456: Departs 2:15 PM, arrives 2:15 AM (+1 day), $380  
- Flight DL789: Departs 6:30 PM, arrives 6:30 AM (+1 day), $420"""

async def book_flight(params: Dict[str, Union[str, int]]) -> str:
    """Book a specific flight."""
    flight_id = params.get("flight_id", "Unknown")
    seat_class = params.get("seat_class", "economy")
    
    confirmation = f"CONF-{flight_id}"
    return f"""Flight booked successfully!
- Flight: {flight_id}
- Class: {seat_class}
- Confirmation: {confirmation}"""

# Define tool specifications for the AI
search_flights_tool = ToolDefinition(
    name="search_flights",
    description="Search for flights from origin to destination",
    arguments=[
        ToolArgument("origin", "string", "Departure city/airport"),
        ToolArgument("destination", "string", "Arrival city/airport"),
        ToolArgument("date", "string", "Travel date")
    ]
)

book_flight_tool = ToolDefinition(
    name="book_flight",
    description="Book a specific flight",
    arguments=[
        ToolArgument("flight_id", "string", "Flight identifier"),
        ToolArgument("seat_class", "string", "Seat class (economy/business/first)")
    ]
)

AVAILABLE_TOOLS = {
    "search_flights": search_flights_tool,
    "book_flight": book_flight_tool
}

print("Tools defined and ready!")

## Step 3: Create AI Decision-Making Activities

These Activities use an LLM to make decisions about which tools to use.

In [ ]:
from temporalio import activity
import json

# Mock LLM call for demonstration (replace with actual LLM integration)
def mock_llm_decision(goal: str, tools: dict, context: str) -> dict:
    """Simulates LLM decision-making."""
    # In a real implementation, this would call OpenAI/Claude/etc.
    if "book" in goal.lower() and "CONF" not in context:
        if "Found 3 flights" in context:
            return {"tool": "book_flight", "parameters": {"flight_id": "UA456", "seat_class": "economy"}}
        return {"tool": "search_flights", "parameters": {"origin": "NYC", "destination": "London", "date": "tomorrow"}}
    return {"tool": "DONE", "parameters": {}}

@activity.defn
async def agent_validate_prompt(agent_goal: AgentGoal, user_prompt: str) -> bool:
    """Validate that the user's request aligns with the agent's capabilities."""
    # Check if the request is related to travel booking
    travel_keywords = ["flight", "book", "travel", "fly", "trip"]
    return any(keyword in user_prompt.lower() for keyword in travel_keywords)

@activity.defn
async def ai_select_tool_with_params(
    goal: str,
    available_tools: dict,
    context: str
) -> dict:
    """AI decides which tool to use next."""
    # In production, this would call your LLM
    decision = mock_llm_decision(goal, available_tools, context)
    print(f"AI Decision: {decision}")
    return decision

print("AI decision-making activities created!")

## Step 4: Create Tool Execution Activities

Each tool becomes a Temporal Activity for reliable execution.

In [ ]:
@activity.defn(name="search_flights")
async def search_flights_activity(params: dict) -> str:
    """Activity wrapper for search_flights tool."""
    return await search_flights(params)

@activity.defn(name="book_flight")
async def book_flight_activity(params: dict) -> str:
    """Activity wrapper for book_flight tool."""
    return await book_flight(params)

print("Tool activities created!")

## Step 5: Build the Agentic Workflow

This is where the magic happens - the agentic loop!

In [ ]:
from datetime import timedelta
from temporalio import workflow

@workflow.defn(sandboxed=False)  # sandboxed=False for Jupyter only
class AgenticWorkflow:
    """Workflow implementing an agentic loop."""

    @workflow.run
    async def run(self, agent_goal: AgentGoal) -> BookingResult:
        """Main agentic loop - AI decides and executes autonomously."""
        
        context = ""
        steps_taken = []
        max_iterations = 10

        print(f"\nStarting agentic loop for: {agent_goal.description}\n")

        # Main agentic loop
        for iteration in range(max_iterations):
            print(f"Iteration {iteration + 1}:")

            # Validate the goal (first iteration only)
            if iteration == 0:
                is_valid = await workflow.execute_activity(
                    "agent_validate_prompt",
                    args=[agent_goal, agent_goal.description],
                    start_to_close_timeout=timedelta(seconds=30),
                )
                
                if not is_valid:
                    return BookingResult(
                        message="Request outside agent capabilities",
                        steps_taken=["Validation failed"]
                    )

            # AI decides which tool to use
            decision = await workflow.execute_activity(
                "ai_select_tool_with_params",
                args=[agent_goal.description, AVAILABLE_TOOLS, context],
                start_to_close_timeout=timedelta(seconds=30),
            )

            selected_tool = decision.get("tool", "")
            parameters = decision.get("parameters", {})

            # Check if done
            if selected_tool.upper() == "DONE":
                print("AI determined goal is complete!\n")
                break

            # Validate tool exists
            if selected_tool not in AVAILABLE_TOOLS:
                print(f"Unknown tool: {selected_tool}\n")
                continue

            # Execute the selected tool
            print(f"Executing: {selected_tool} with params: {parameters}")
            
            result = await workflow.execute_activity(
                selected_tool,
                args=[parameters],
                start_to_close_timeout=timedelta(seconds=30),
            )

            # Update context and track steps
            context += f"\n\nExecuted: {selected_tool}\nResult: {result}"
            steps_taken.append(f"{selected_tool}: {result[:100]}...")
            print(f"Result: {result[:100]}...\n")

        # Prepare final result
        if not steps_taken:
            steps_taken = ["No actions were taken"]

        success = any("book_flight" in step for step in steps_taken)
        message = f"{'Successfully completed' if success else 'Partially completed'}: {agent_goal.description}"

        return BookingResult(
            message=message,
            steps_taken=steps_taken
        )

print("Agentic workflow defined!")

## Step 6: Run the Worker

The Worker executes our Workflow and Activities.

In [ ]:
from temporalio.client import Client
from temporalio.worker import Worker
import concurrent.futures
import asyncio

async def run_worker():
    client = await Client.connect("localhost:7233")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        worker = Worker(
            client,
            task_queue="agentic-queue",
            workflows=[AgenticWorkflow],
            activities=[
                agent_validate_prompt,
                ai_select_tool_with_params,
                search_flights_activity,
                book_flight_activity,
            ],
            activity_executor=executor,
        )
        print("Worker started and listening for tasks...")
        await worker.run()

# For Jupyter notebook execution
# worker_task = asyncio.create_task(run_worker())
print("Worker ready to start")

## Step 7: Execute the Agent

Now let's create an agent goal and watch it work!

In [ ]:
async def execute_agent():
    client = await Client.connect("localhost:7233")
    
    # Create the agent's goal
    agent_goal = AgentGoal(
        agent_name="Travel Booking Assistant",
        tools=[search_flights_tool, book_flight_tool],
        description="Book a flight from NYC to London for tomorrow",
        starter_prompt="You are an expert travel agent.",
        example_conversation_history="User: I need a flight\nAgent: I'll help you find flights."
    )
    
    print("Starting agentic workflow...")
    
    handle = await client.start_workflow(
        AgenticWorkflow.run,
        agent_goal,
        id="travel-agent-workflow",
        task_queue="agentic-queue",
    )
    
    result = await handle.result()
    
    print("\n" + "="*50)
    print("FINAL RESULT:")
    print("="*50)
    print(result.message)
    print("\nSteps taken:")
    for i, step in enumerate(result.steps_taken, 1):
        print(f"  {i}. {step}")

# Execute the agent
# await execute_agent()
print("Agent execution function ready")

## Understanding the Agentic Loop

Let's visualize what's happening in our agent:

In [ ]:
import base64
from IPython.display import Image, display

def render_mermaid(graph_definition):
    graph_bytes = graph_definition.encode("ascii")
    base64_bytes = base64.b64encode(graph_bytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

diagram = """
graph TD
    A[User Request] --> B[Validate Request]
    B --> C{Valid?}
    C -->|No| D[Return Error]
    C -->|Yes| E[Start Loop]
    E --> F[AI Selects Tool]
    F --> G{Tool = DONE?}
    G -->|Yes| H[Return Result]
    G -->|No| I[Execute Tool]
    I --> J[Update Context]
    J --> F
    
    style E fill:#e1f5fe
    style F fill:#fff3e0
    style I fill:#e8f5e8
"""

render_mermaid(diagram)

## Key Benefits of This Approach

### 1. **Reliability**
- Activities automatically retry on failure
- State persists across crashes
- Workflow continues from last checkpoint

### 2. **Observability**
- Every decision is logged
- Tool executions are tracked
- Full audit trail in Temporal Web UI

### 3. **Flexibility**
- Agent adapts based on context
- Tools can be added/removed easily
- Different agents for different domains

### 4. **Cost Efficiency**
- No repeated LLM calls after failures
- Efficient resource usage
- Predictable execution costs

## Advanced Patterns

### Human-in-the-Loop
Add signals to pause for human approval:
```python
await workflow.wait_condition(lambda: self.human_approved)
```

### Multi-Agent Orchestration
Chain multiple specialized agents:
```python
research_result = await workflow.execute_child_workflow(
    ResearchAgentWorkflow.run, research_goal
)
booking_result = await workflow.execute_child_workflow(
    BookingAgentWorkflow.run, booking_goal
)
```

### Long-Running Agents
Use continue-as-new for infinite loops:
```python
if iteration >= 100:
    workflow.continue_as_new(updated_state)
```

## Summary

We've built a durable AI agent that:
1. **Makes intelligent decisions** using an LLM
2. **Executes tools dynamically** based on context
3. **Maintains state** across failures
4. **Provides full observability** of its actions
5. **Handles failures gracefully** with automatic retries

This pattern transforms unreliable AI interactions into robust, production-ready systems!

### Next Steps
- Integrate with real LLM providers (OpenAI, Claude, etc.)
- Add more sophisticated tools
- Implement human-in-the-loop patterns
- Deploy to production with Temporal Cloud

## Exercise: Build Your Own Agent

Try modifying the code to:
1. Add a new tool (e.g., `check_weather` or `send_email`)
2. Change the agent's goal to use multiple tools
3. Add error handling for specific tool failures
4. Implement a retry policy with exponential backoff

**Challenge:** Can you make the agent handle a complex multi-step booking process?